# Import Functional Data
Import CSV data into a functional attribute and update the parameters and header units.

## Import required libraries
Connect to MI and specify a table.

In [1]:
from datetime import datetime
import csv
from GRANTA_MIScriptingToolkit import granta as mpy

mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)
db = mi.get_db(db_key='MI_Training')
tab = db.get_table('Design Data')

## Create a new record to store your data
Define names for a new record, and an attribute on that record.

In [2]:
now = datetime.now().strftime("%c")
recordName = 'STK Example 7:{}'.format(now)
funcAttributeName = 'Tensile Stress/Strain, L'

Create the new **Record** object.

In [3]:
rec = tab.create_record(recordName, subsets={'Design Data'})
rec

<Record long name:STK Example 7:Thu Jul 15 10:43:33 2021>

Get the unpopulated functional attribute for the new record, so we can populate it.

In [4]:
func = rec.attributes[funcAttributeName]
func

<FloatFunctionalSeriesValue name: Tensile Stress/Strain, L, with 0 points>

## Import test data
The tensile test data to import is stored in `example.csv`.
Read in each row of the CSV data, and add the data to the functional attribute point-by-point.

In [5]:
with open('example.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        func.add_point({'y': float(row['Tensile Stress [ksi]']),
                        'Strain': float(row['Strain [%]']),
                        'Temperature': float(row['Temperature [Celsius]']),
                        'Stress/Strain Curve Type': str(row['Curve Type']),
                        'Estimated?': bool(row['Estimated?'])})

Access the `func.data` property to see the underlying data structure of the attribute represented as a list of lists.
`func.data` shows all parameters supported by the attribute. Parameters that were not populated during the import have
been set to **None**.

In [6]:
func.data

[['Y min (Tensile Stress/Strain, L [ksi])',
  'Y max (Tensile Stress/Strain, L [ksi])',
  'Strain [% strain]',
  'Temperature [K]',
  'Time [hr]',
  'Other []',
  'Stress/Strain Curve Type []',
  'Estimated point?'],
 [0.0, 0.0, 0.0, -73.0, None, None, 'Yield', True],
 [396.0, 396.0, 0.209, -73.0, None, None, 'Yield', True],
 [714.0, 714.0, 0.376, -73.0, None, None, 'Yield', True],
 [1031.0, 1031.0, 0.544, -73.0, None, None, 'Yield', True],
 [1269.0, 1269.0, 0.669, -73.0, None, None, 'Yield', True],
 [1427.0, 1427.0, 0.753, -73.0, None, None, 'Yield', True],
 [1507.0, 1507.0, 0.795, -73.0, None, None, 'Yield', True],
 [1570.0, 1570.0, 0.83, -73.0, None, None, 'Yield', True],
 [1633.0, 1633.0, 0.867, -73.0, None, None, 'Yield', True],
 [1681.0, 1681.0, 0.897, -73.0, None, None, 'Yield', True],
 [1721.0, 1721.0, 0.925, -73.0, None, None, 'Yield', True],
 [1752.0, 1752.0, 0.952, -73.0, None, None, 'Yield', True],
 [1784.0, 1784.0, 0.983, -73.0, None, None, 'Yield', True],
 [1808.0, 1808.0

The example data uses Celsius for temperature, but the database default is Kelvin.
Change the parameter unit before import to take account of this.

In [7]:
func.parameters['Temperature'].unit = '°C'

After changing the units, it is good practice to update the header units in `func.data` so that the units displayed in
MI applications are correct.

In [8]:
func.update_header_units()

## Write your changes to MI

In [9]:
rec.set_attributes([func])
mi.update([rec])

[<Record long name:STK Example 7:Thu Jul 15 10:43:33 2021>]